<a href="https://colab.research.google.com/github/berholz/ads_test_task/blob/main/test_%D0%92%D0%B8%D0%BA%D1%82%D0%BE%D1%80%D0%B8%D1%8F_%D0%91%D0%B5%D1%80%D1%85%D0%BE%D0%BB%D1%8C%D1%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Добрый день!
Меня зовут Виктория и это мое тестовое задание

Я создала 2 функции для построения витрины: одна просто суммирует нужные значения за выбранный период, вторая -делает ресемплирования и можно отслеживать данные по выбранному периоду времени-день, час, полчаса etc.
Формат времени в датафреме UTC, поэтому мне не удалось выудить часовые пояса, к  сожалению

In [82]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from IPython.display import display


from datetime import datetime, timedelta, timezone
from tzlocal import get_localzone # $ pip install tzlocal


Здесь происходит считывание данных. У меня таблицы лежали в Google disk

In [83]:
from google.colab import drive
drive.mount('/content/drive/')
path = "drive/My Drive/ad_sys.csv"
ad_sys= pd.read_csv(path)
path = "drive/My Drive/billing_sys.csv"
billing_sys= pd.read_csv(path)
path = "drive/My Drive/internal_events.csv"
internal_events= pd.read_csv(path)
path = "drive/My Drive/uid_to_user_id.csv"
uid_to_user_id= pd.read_csv(path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Названия колонок из ТЗ
**file: ad_sys.csv**

date- дата отчетного периода

_collected_at- время и дата получения данных на сервере

revenue- данные по выручке от рекламной системы

spent- данные по затратам от рекламной системы

affid- идентификатор источника трафика

**file: billing_sys.csv**

created_at- время и дата транзакции

affid- идентификатор источника трафика

amount- сумма транзакции

uid- идентификатор пользователя в биллинг системе

Внутренняя система событий по покупкам 

**file: internal_events.csv**

created_at- время и дата события

affid- дентификатор источника трафика

revenue- сумма транзакции

user_id- внутренний идентификатор пользователя

 
 
 
 
Необходимо сделать представление / view / витрину, по которой в дальнейшем будет строится отчеты с данными следующего вида:

revenue_traffic_source- выручка по данным рекламной системы

date - дата

revenue_pixel- выручка по данным внутренней системы событий

revenue_billing_system- выручка по данным системы биллинга

revenue_{traffic source id}- выручка по конкретному источнику трафика

spent_traffic_source- данные по затратам на рекламу



In [84]:
#ad_sys.head(10)

In [85]:
ad_sys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           775 non-null    object 
 1   revenue        775 non-null    float64
 2   spent          775 non-null    float64
 3   _collected_at  775 non-null    object 
 4   affid          775 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 30.4+ KB


Здесь я привожу столбец времени к формату datetime, чтобы выполнять фильтрацию по времени. Столбец со временим называется original time, и он становитс индексом для всего датафрема(Для всех датафреймов будет выполнена такая же операция)


In [86]:
ad_sys['original_date']=pd.to_datetime(ad_sys['_collected_at'])
ad_sys=ad_sys.set_index('original_date')
ad_sys=ad_sys.sort_index()
print(ad_sys.index.is_monotonic)

True


In [87]:
ad_sys['affid']=ad_sys['affid'].astype(np.float64)
ad_sys['affid']=ad_sys['affid'].astype(object)

In [88]:
billing_sys.head(5)

,created_at,affid,amount,uid
0,2021-04-08 12:27:49 UTC,1008.0,99.0,bf00c161
1,2021-04-11 05:49:51 UTC,1210.0,39.0,c7dc59fc
2,2021-04-09 01:30:36 UTC,1999.0,39.0,da121a2a
3,2021-04-08 03:06:28 UTC,1999.0,39.0,244c8870
4,2021-04-10 20:55:09 UTC,NaN,39.0,f07d2882


In [89]:
billing_sys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   created_at  1125 non-null   object 
 1   affid       973 non-null    float64
 2   amount      1125 non-null   float64
 3   uid         1125 non-null   object 
dtypes: float64(2), object(2)
memory usage: 35.3+ KB


In [90]:
billing_sys['original_date']=pd.to_datetime(billing_sys['created_at'])
billing_sys=billing_sys.set_index('original_date')
billing_sys=billing_sys.sort_index()
print(billing_sys.index.is_monotonic)

True


In [91]:
internal_events.head(5)

,created_at,affid,revenue,user_id
0,2021-04-08 02:46:32.319760 UTC,NaN,39.0,15232625
1,2021-04-08 14:45:35.272310 UTC,1210.0,39.0,15235225
2,2021-04-08 14:59:56.842434 UTC,1009.0,39.0,15235207
3,2021-04-08 16:05:28.244022 UTC,1210.0,39.0,3719420
4,2021-04-08 19:25:10.870616 UTC,1008.0,39.0,15236390


In [92]:
internal_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   created_at  1459 non-null   object 
 1   affid       1343 non-null   float64
 2   revenue     1459 non-null   float64
 3   user_id     1459 non-null   object 
dtypes: float64(2), object(2)
memory usage: 45.7+ KB


In [93]:
internal_events['affid']=internal_events['affid'].fillna('unknown')
billing_sys['affid']=billing_sys['affid'].fillna('unknown')
ad_sys['affid']=ad_sys['affid'].fillna('unknown')

In [94]:
internal_events['original_date']=pd.to_datetime(internal_events['created_at'])
internal_events=internal_events.set_index('original_date')
internal_events=internal_events.sort_index()
print(internal_events.index.is_monotonic)

True


In [95]:
uid_to_user_id.head()

,uid,user_id
0,ab99a80e,15240245
1,f7c8256c,15235689
2,6b7dd846,15233251
3,685c6868,15240351
4,54314e60,15243647


In [96]:
uid_to_user_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   uid      643 non-null    object
 1   user_id  643 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ KB


**Задание из ТЗ такое:**

Необходимо сделать представление / view / витрину, по которой в дальнейшем будет строится отчеты с данными следующего вида:

revenue_traffic_source- выручка по данным рекламной системы

date - дата

revenue_pixel- выручка по данным внутренней системы событий

revenue_billing_system- выручка по данным системы биллинга

revenue_{traffic source id}- выручка по конкретному источнику трафика

spent_traffic_source- данные по затратам на рекламу

При этом должна сохраниться возможность фильтрации по временной зоне (timezone).


Первая функция **Функция revenue_summ** создает витрину и возвращет ее в качестве dataframe со следующими аргументами:

date1- начало отчетного периода

date2-конец отчетного периода

affid- id источника трафика

info(True/False)-true: печатает дополнительную информацию: 1. revenue для конретного affid по разным источникам
2. revenue по всем affid для сравнения

In [102]:
def revenue_summ(date1,date2,affid, info):
  print('Summ of revenue from ad, billing and internal sources',date1,'-',date2)
  result_df=pd.DataFrame(columns=['date', 'revenue_traffic_source','revenue_pixel','revenue_billing_system', 'revenue_{}'.format(affid),'spent_traffic_source'])
  pivot=ad_sys[date1:date2]
  #print(pivot.head(5))
  pivot_ad=pivot.pivot_table(index='affid',values=['revenue'], aggfunc='sum')
  print()
  #display(pivot_ad)
  #print(pivot['revenue'].sum())
  result_df.loc[0,'date']=(date1,date2)
  result_df.loc[0,'revenue_traffic_source']=pivot['revenue'].sum()
  
  money_spent=pivot['spent'].sum()
  pivot=billing_sys[date1:date2]
  result_df.loc[0,'revenue_billing_system']=pivot['amount'].sum()
  pivot_bill=pivot.pivot_table(index='affid',values='amount', aggfunc='sum')
  #display(pivot_bill)
  pivot=internal_events[date1:date2]
  result_df.loc[0,'revenue_pixel']=pivot['revenue'].sum()
  result_df.loc[0,'spent_traffic_source']=money_spent
  pivot_internal=pivot.pivot_table(index='affid',values='revenue', aggfunc='sum')
  #display(pivot_internal)
  
  data_merge = pivot_bill.merge(pivot_internal, on='affid', how='left')
  data_merge.reset_index(inplace=True)
  pivot_ad.reset_index(inplace=True)
  pivot_ad['affid']=pivot_ad['affid'].astype(object)
  data_merge1 = data_merge.merge(pivot_ad, on='affid', how='left')
  data_merge1.columns=['affid','billing_system', 'pixel','traffic_source']
  try:
    k=int(data_merge1.loc[data_merge1['affid']==affid,'traffic_source'])
  except:
    k=None
  result_df.loc[0,'revenue_{}'.format(affid)]=k
  #display(result_df)
  if info== True:
      print('revenue by affid=',affid)
      print()
      display(data_merge1.loc[data_merge1['affid']==affid])
      print()
      print('Money spent on ad=', money_spent)
      print()
      print('revenue from all affids')
      print()
      display(data_merge1)
  return(result_df)
  

Примеры работы функции **revenue_summ**

In [98]:
revenue_summ('2021-04-08','2021-04-21',1008, info=True)

Summ of revenue from ad, billing and internal sources 2021-04-08 - 2021-04-21

revenue by affid= 1008



,affid,billing_system,pixel,traffic_source
1,1008,16598.91,17159.87,2490968.35



Money spent on ad= 3786685.63

revenue from all affids



,affid,billing_system,pixel,traffic_source
0,1000,259.00,298.00,NaN
1,1008,16598.91,17159.87,2490968.35
2,1009,1789.01,1788.99,NaN
3,1012,39.00,39.00,NaN
4,1026,39.00,39.00,NaN
5,1042,177.00,177.00,NaN
6,1209,1971.38,1971.38,NaN
7,1210,9192.47,9192.38,NaN
8,1211,1979.00,2116.33,NaN
9,1238,95.00,95.00,NaN


,date,revenue_traffic_source,revenue_pixel,revenue_billing_system,revenue_1008,spent_traffic_source
0,"(2021-04-08, 2021-04-21)",2.49097e+06,103500,52600.1,2490968,3.78669e+06


In [99]:
revenue_summ('2021-04-08','2021-04-21',1008, info=False)

Summ of revenue from ad, billing and internal sources 2021-04-08 - 2021-04-21



,date,revenue_traffic_source,revenue_pixel,revenue_billing_system,revenue_1008,spent_traffic_source
0,"(2021-04-08, 2021-04-21)",2.49097e+06,103500,52600.1,2490968,3.78669e+06


In [103]:
revenue_summ('2021-04-10','2021-04-20',1009, info=False)

Summ of revenue from ad, billing and internal sources 2021-04-10 - 2021-04-20



,date,revenue_traffic_source,revenue_pixel,revenue_billing_system,revenue_1009,spent_traffic_source
0,"(2021-04-10, 2021-04-20)",2.12972e+06,63513.4,28453,None,3.21026e+06


In [104]:
revenue_summ('2021-04-10 00:00:00','2021-04-20 15:00:00',1009, info=False)

Summ of revenue from ad, billing and internal sources 2021-04-10 00:00:00 - 2021-04-20 15:00:00



,date,revenue_traffic_source,revenue_pixel,revenue_billing_system,revenue_1009,spent_traffic_source
0,"(2021-04-10 00:00:00, 2021-04-20 15:00:00)",2.12972e+06,63513.4,28453,None,3.21026e+06


Вторая функция:
Если необходимо сделать витрину и ресемплирование в отчетном периоде по часу,дню или неделе, то создадим другую функцию **revenue_resample**. Функция возвращает dataframe Аргументы следующие:

date1-начало отсчетного периода

date2-конец отсчетного периода

period- период ресемплирования, например 1H- 1 час, '30min'-30 минут

affid-id источника трафика

In [ ]:
def revenue_resample(date1,date2, period, affid):
  ad_series=ad_sys.loc[date1:date2,'revenue'].resample(period).sum()
  ad_spent=ad_sys.loc[date1:date2,'spent'].resample(period).sum()
  ad_affid=ad_sys[ad_sys['affid']==affid]
  ad_affid_series=ad_affid.loc[date1:date2,'revenue'].resample(period).sum()
  bill_series=billing_sys.loc[date1:date2,'amount'].resample(period).sum()
  internal_series=internal_events.loc[date1:date2,'revenue'].resample(period).sum()
  merged=pd.merge_asof(bill_series,ad_series, on='original_date')
  merged1=pd.merge_asof(merged, internal_series, on='original_date')
  merged2=pd.merge_asof(merged1, ad_spent, on='original_date')
  merged3=pd.merge_asof(merged2, ad_affid_series, on='original_date')
  #merged3.reset_index(inplace=True)
  merged3.columns=['date', 'revenue_billing_system','revenue_traffic_source','revenue_pixel','spent_traffic_source', 'revenue_{}'.format(affid)]
  #print(merged3)
  return(merged3)

Примеры работы функции** revenue_resample**

In [105]:
revenue_resample('2021-04-08','2021-04-21','1H',1008)

,date,revenue_billing_system,revenue_traffic_source,revenue_pixel,spent_traffic_source,revenue_1008
0,2021-04-08 00:00:00+00:00,56.00,NaN,56.00,NaN,NaN
1,2021-04-08 01:00:00+00:00,294.00,NaN,294.00,NaN,NaN
2,2021-04-08 02:00:00+00:00,134.00,NaN,256.28,NaN,NaN
3,2021-04-08 03:00:00+00:00,428.00,NaN,565.35,NaN,NaN
4,2021-04-08 04:00:00+00:00,394.40,NaN,406.03,NaN,NaN
...,...,...,...,...,...,...
91,2021-04-11 19:00:00+00:00,415.02,24464.66,1760.93,35150.78,24464.66
92,2021-04-11 20:00:00+00:00,539.99,36696.99,699.95,52828.05,36696.99
93,2021-04-11 21:00:00+00:00,451.98,36696.99,891.95,52898.63,36696.99
94,2021-04-11 22:00:00+00:00,255.96,24464.66,255.96,35284.34,24464.66


In [106]:
revenue_resample('2021-04-08','2021-04-20','2H',1008)

,date,revenue_billing_system,revenue_traffic_source,revenue_pixel,spent_traffic_source,revenue_1008
0,2021-04-08 00:00:00+00:00,350.00,NaN,350.00,NaN,NaN
1,2021-04-08 02:00:00+00:00,562.00,NaN,821.63,NaN,NaN
2,2021-04-08 04:00:00+00:00,623.40,NaN,674.03,NaN,NaN
3,2021-04-08 06:00:00+00:00,287.99,0.00,287.99,57.65,0.00
4,2021-04-08 08:00:00+00:00,134.00,0.00,573.97,1077.29,0.00
5,2021-04-08 10:00:00+00:00,923.99,765.00,1523.88,4219.34,765.00
6,2021-04-08 12:00:00+00:00,1567.99,2746.99,3166.34,11789.45,2746.99
7,2021-04-08 14:00:00+00:00,2640.59,8031.13,4246.33,19517.55,8031.13
8,2021-04-08 16:00:00+00:00,1874.43,12643.90,3279.89,23215.97,12643.90
9,2021-04-08 18:00:00+00:00,460.97,16104.48,1379.86,29105.93,16104.48


In [107]:
revenue_resample('2021-04-08','2021-04-20','1D',1009)

,date,revenue_billing_system,revenue_traffic_source,revenue_pixel,spent_traffic_source,revenue_1009
0,2021-04-08 00:00:00+00:00,11408.36,66020.72,18649.17,133559.99,NaN
1,2021-04-09 00:00:00+00:00,12738.81,295224.09,21337.16,442861.02,NaN
2,2021-04-10 00:00:00+00:00,13025.20,329364.39,29908.02,459717.83,NaN
3,2021-04-11 00:00:00+00:00,15427.76,581852.84,33605.35,843381.80,NaN


Спасибо за интересное задание, надеюсь на дальнешее сотрдничество.
Виктория